In [1]:
using CSV
using DataFrames
using Serialization
using HDF5

In [2]:
df = DataFrame(CSV.File("top20_adjacency_mat.csv"));
population_sizes = Dict(
    "PEK" => 16_938_000,
    "LHR" => 9_800_000,
    "CGK" => 9_733_000,
    "DEL" => 16_787_941,
    "BOM" => 12_442_373,
    "ATL" => 4_661_000,
    "HND" => 36_975_000,
    "BKK" => 8_484_000,
    "CDG" => 10_514_000,
    "CAN" => 10_546_000,
    "PVG" => 20_948_000,
    "SIN" => 5_176_000,
    "LAX" => 12_197_000,
    "KUL" => 6_005_000,
    "CTU" => 7_570_000,
    "AMS" => 1_073_000,
    "GRU" => 11_310_000,
    "DEN" => 2_428_000,
    "HKG" => 7_066_000,
    "MNL" => 12_075_689
);


In [3]:
df[df.Origin .== "AMS",:]

Row,Origin,AMS,ATL,BKK,BOM,CAN,CDG,CGK,CTU,DEL,DEN,GRU,HKG,HND,KUL,LAX,LHR,MNL,PEK,PVG,SIN
,String3,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AMS,0.0,1.49482e5,1.9045e5,89757.0,2552.79,1.79445e5,1161.55,42272.0,91847.4,1315.21,47736.9,2.02192e5,131.195,91241.8,101692.0,2.77285e5,73382.7,206660.0,170692.0,1.44559e5


In [13]:
# this adjacency matrix will store the number of migrations per origin population in a given year
new_adjacency_matrix = Array{Float64}(undef,size(df,1), size(df,2)-1)
population_sizes_vector = Vector{Int64}(undef,length(population_sizes))

for from_idx = 1:size(new_adjacency_matrix,2)
    for to_idx = 1:size(new_adjacency_matrix,1)
        new_adjacency_matrix[to_idx,from_idx] = df[from_idx,to_idx+1] / population_sizes[df[from_idx,1]]
    end
    # record the population size for the corresponding col of the migration matrix
    population_sizes_vector[from_idx] = population_sizes[df[from_idx,1]] 
end

# Currently, this is the rate per year, we want the rate in units of the recovery time (typically 7 days, 1 week) so divide by 52
new_adjacency_matrix ./= 52


dfnew = DataFrame(new_adjacency_matrix,:auto)
dfnew[!, :population_sizes] = population_sizes_vector  # Append as a new column

CSV.write("cleaned_adjacency_matrix.csv", dfnew)
# serialize("cleaned_adjacency_matrix", (new_adjacency_matrix, population_sizes_vector))
# h5write("cleaned_adjacency_matrix.h5", "matrix", new_adjacency_matrix)
# h5write("cleaned_adjacency_matrix.h5", "populations", population_sizes_vector)

KeyError: KeyError: key 0.0 not found

In [15]:
df = CSV.read("cleaned_adjacency_matrix.csv", DataFrame)
new_adjacency_matrix2 = Matrix(df[:, 1:end-1])  # Assuming the last column is the population sizes vector
population_sizes_vector = df[:,end]

20-element Vector{Int64}:
  1073000
  4661000
  8484000
 12442373
 10546000
 10514000
  9733000
  7570000
 16787941
  2428000
 11310000
  7066000
 36975000
  6005000
 12197000
  9800000
 12075689
 16938000
 20948000
  5176000

In [11]:
dfnew

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,population_sizes
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,0.0,0.000580608,0.000442325,0.000138727,5.04122e-6,0.000332871,2.34489e-6,0.000107387,0.000112242,1.03649e-5,7.90267e-5,0.000539366,9.03414e-8,0.000289843,0.000160336,0.000577788,0.000122313,0.000234318,0.000156699,0.000554007,1073000
2,0.00267908,0.0,7.40452e-6,2.65045e-6,1.71958e-6,0.000366062,5.58945e-7,6.51198e-7,2.15559e-6,0.00658377,9.83602e-5,6.66137e-6,2.49465e-8,2.24321e-6,0.00103496,0.000336637,1.46029e-6,8.18442e-6,2.74399e-6,1.18271e-5,4661000
3,0.00341333,1.17839e-5,0.0,0.000542538,0.00089651,0.000354342,0.000454973,0.000108996,0.000496889,1.80453e-6,1.93987e-6,0.00120577,5.07143e-7,0.000991733,0.000102374,0.000507654,0.000684444,0.000329259,0.000318902,0.00223587,8484000
4,0.00160866,5.71466e-6,0.000803773,0.0,8.90276e-6,0.000169829,1.10225e-5,8.96403e-6,0.00408188,1.45797e-6,1.34063e-6,0.000739342,1.48483e-7,0.000329629,3.27718e-6,0.000425612,9.0676e-6,9.21172e-6,6.16029e-6,0.00159625,12442373
5,4.57523e-5,3.54992e-6,0.00118423,9.34151e-6,0.0,0.000161221,0.000196777,0.00219985,4.78824e-5,6.98981e-7,3.30487e-7,0.0,3.65157e-7,0.000752827,9.62216e-5,4.70526e-6,8.01069e-5,0.00156836,0.000150214,0.00107726,10546000
6,0.00321608,0.000783349,0.000500578,0.000143429,0.000161264,0.0,2.79957e-6,6.55347e-6,0.000157916,8.71759e-6,9.30349e-5,0.000413633,4.01748e-8,0.000292443,0.000400628,0.000652516,6.30478e-6,0.000325991,0.000195858,0.000720303,10514000
7,2.08178e-5,1.03663e-6,0.000521953,6.9511e-7,0.000181319,2.04307e-6,0.0,2.2997e-5,1.18392e-6,0.0,2.56247e-7,0.00058552,2.02613e-7,0.000660333,2.74761e-6,3.17492e-6,7.74633e-5,4.34889e-5,2.40596e-5,0.00163804,9733000
8,0.000757617,8.26367e-7,9.72532e-5,5.45376e-6,0.00157813,3.26242e-6,1.78863e-5,0.0,4.86832e-6,1.09961e-8,1.22616e-7,0.000627409,4.89967e-7,0.000255121,1.76438e-6,4.10119e-6,9.0676e-6,0.00214578,0.000862392,0.00024173,7570000
9,0.00164613,5.76783e-6,0.000897723,0.00527918,7.62228e-5,0.000252147,1.07169e-5,1.08011e-5,0.0,2.56538e-6,1.75936e-6,0.000793042,3.17584e-7,0.000343656,4.21622e-6,0.000432028,9.0676e-6,7.78348e-5,9.64504e-5,0.00112351,16787941


In [18]:
df

Row,Origin,AMS,ATL,BKK,BOM,CAN,CDG,CGK,CTU,DEL,DEN,GRU,HKG,HND,KUL,LAX,LHR,MNL,PEK,PVG,SIN
,String3,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AMS,0.0,1.49482e5,1.9045e5,89757.0,2552.79,1.79445e5,1161.55,42272.0,91847.4,1315.21,47736.9,2.02192e5,131.195,91241.8,101692.0,2.77285e5,73382.7,206660.0,170692.0,1.44559e5
2,ATL,1.40723e5,0.0,2856.1,1385.07,860.401,1.89862e5,251.249,200.288,1397.96,794270.0,55088.6,2245.84,37.3468,631.663,6.5607e5,171550.0,573.426,6488.61,2694.77,2662.23
3,BKK,1.9514e5,3266.64,0.0,354599.0,522445.0,220839.0,230269.0,42905.0,3.96046e5,287.333,1112.95,4.47604e5,821.049,3.12298e5,65145.0,2.94171e5,392220.0,290003.0,352779.0,5.74536e5
4,BOM,89757.0,1714.85,351024.0,0.0,6043.99,92799.0,449.739,3528.6,3.41565e6,258.813,857.48,271658.0,278.04,102930.0,2557.32,2.56712e5,5690.02,8124.39,6710.38,4.27129e5
5,CAN,2764.57,943.002,4.91639e5,4882.2,0.0,88436.0,99434.0,865435.0,41800.0,100.229,170.751,0.0,462.087,2.18577e5,61028.0,2570.29,50302.0,1.34316e6,163628.0,289948.0
6,CDG,1.8199e5,2.00136e5,1.93729e5,92850.0,88144.0,0.0,1117.0,1783.66,137856.0,1116.63,49692.9,144179.0,55.5344,83736.7,2.55414e5,3.11779e5,2537.42,287340.0,1.93404e5,1.6244e5
7,CGK,1186.78,282.891,230269.0,5578.65,99592.0,1416.91,0.0,9052.52,5423.99,1.11253,159.218,279656.0,413.434,2.53377e5,2315.65,1972.77,48336.0,38052.0,37296.0,5.33552e5
8,CTU,42272.0,256.337,42905.0,3528.6,865947.0,2579.71,9052.52,0.0,4251.76,1.5718,81.8455,230530.0,830.77,79664.0,1438.72,2894.34,5690.02,1.88979e6,949444.0,65062.0
9,DEL,97984.4,1881.77,4.3377e5,3.56337e6,41800.0,137856.0,1033.53,4249.91,0.0,520.992,1287.4,291645.0,683.035,107310.0,3584.14,2.8144e5,5690.02,68330.0,104763.0,302395.0


In [23]:
migration_matrix , popsiz= deserialize("cleaned_adjacency_matrix")

([0.0 0.0005806082356047729 … 0.00015669937280592233 0.0005540071350760907; 0.002679083764248333 0.0 … 2.7439891290337976e-6 1.1827096922185234e-5; … ; 0.003059215714388128 1.1118315529434092e-5 … 0.0 0.0025055561532145404; 0.002590854407125959 1.0984055958609081e-5 … 0.0005722101866802045 0.0], [1073000, 4661000, 8484000, 12442373, 10546000, 10514000, 9733000, 7570000, 16787941, 2428000, 11310000, 7066000, 36975000, 6005000, 12197000, 9800000, 12075689, 16938000, 20948000, 5176000])

In [24]:
popsiz

20-element Vector{Int64}:
  1073000
  4661000
  8484000
 12442373
 10546000
 10514000
  9733000
  7570000
 16787941
  2428000
 11310000
  7066000
 36975000
  6005000
 12197000
  9800000
 12075689
 16938000
 20948000
  5176000